<a href="https://colab.research.google.com/github/MarcioLuizBR/desafio_imersaoIA2025/blob/main/HELPTEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#--coding: utf-8--

# Finalização da Imersão IA 3ª Edição da Alura com Google Gemini


### Projeto "HELPTEA" (Ajuda para familiares de pessoas com Transtornos do Espectro Autista)

Desafio de desenvolvimento de uma aplicação pratica dos conceitos apresentados na imersão Alura/Google Gemini

#### instalação dos pacotes essenciais

Instala as bibliotecas necessárias para uso dos modelos generativos do Google e para execução dos agentes

In [44]:
!pip -q install google-generativeai
!pip -q install google-adk

#### Importação de bibliotecas e configuração da chave da API

In [45]:
import os, textwrap, warnings
import requests # Para fazer requisições HTTP
from google.colab import userdata
from google import generativeai as genai
from datetime import date
from IPython.display import HTML, Markdown, display
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)


warnings.filterwarnings("ignore")

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
MODEL_ID = "gemini-2.0-flash"

#### Inicializa a instância do modelo generativo

Cria sessões de chat com histórico vazio para simular especialistas

In [46]:
from google.generativeai import GenerativeModel
modelo = GenerativeModel("gemini-2.0-flash")

agente_assistenteSocial = modelo.start_chat(history=[])
agente_juridico = modelo.start_chat(history=[])
agente_educacional = modelo.start_chat(history=[])
agente_nutricionista = modelo.start_chat(history=[])
agente_psicologo = modelo.start_chat(history=[])
agente_redator = modelo.start_chat(history=[])


#### Função auxiliar para exibir texto formatado em Markdown no Colab

In [47]:
def to_markdown(text):
    """Exibe Markdown bonito no Colab."""
    text = text.replace('•', '  *')
    display(Markdown(textwrap.indent(text, "> ", lambda _: True)))

#### Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final

In [48]:
def call_agent(agent: Agent, user_message: str, user_id="user1", session_id="s1"):
    """Roda o agente via Runner e devolve a resposta final (string)."""
    sess_service = InMemorySessionService()
    sess_service.create_session(app_name=agent.name, user_id=user_id, session_id=session_id)
    runner = Runner(agent=agent, app_name=agent.name, session_service=sess_service)
    content = types.Content(role="user", parts=[types.Part(text=user_message)])

    final_text = ""
    for ev in runner.run(user_id=user_id, session_id=session_id, new_message=content):
        if ev.is_final_response():
            for p in ev.content.parts:
                if p.text:
                    final_text += p.text + "\n"
    return final_text.strip()

#### Função que define os agentes da aplicação

In [ ]:
def build_agents() -> dict:
    hoje = date.today().strftime("%d/%m/%Y")

    return {
        "assistente_social": Agent(
            name="assistente_social",
            model=MODEL_ID,
            description="Orientação prática sobre benefícios sociais, serviços públicos e ONGs.",
            instruction=f"""
                Você é Assistente Social especializado em TEA.
                • Fale de forma acolhedora e objetiva.
                • Traga exemplos de benefícios (BPC, passe livre, etc.).
                • Data de referência: {hoje}.
            """,
            tools=[google_search]
        ),

        "juridico": Agent(
            name="juridico",
            model=MODEL_ID,
            description="Direitos legais da pessoa autista.",
            instruction=f"""
                Você é advogado especializado em TEA.
                • Cite leis (12.764/2012, LBI 13.146/2015) quando útil.
                • Seja prático e sem jargões excessivos.
                • Data de referência: {hoje}.
            """,
            tools=[google_search]
        ),

        "educacional": Agent(
            name="educacional",
            model=MODEL_ID,
            description="Estratégias pedagógicas inclusivas.",
            instruction="""
                Você é pedagogo especialista em educação inclusiva para TEA.
                • Sugira adaptações simples, rotinas visuais, TEACCH/PECS.
            """,
            tools=[google_search]
        ),

        "nutricionista": Agent(
            name="nutricionista",
            model=MODEL_ID,
            description="Recomendações sobre seletividade alimentar e hábitos saudáveis.",
            instruction="""
                Você é nutricionista especializado em TEA.
                • Dê dicas práticas para ampliar repertório alimentar.
                • Oriente sempre consultar profissional presencial.
            """,
            tools=[google_search]
        ),

        "psicologo": Agent(
            name="psicologo",
            model=MODEL_ID,
            description="Apoio emocional e manejo comportamental.",
            instruction="""
                Você é psicólogo infantil com foco em TEA.
                • Traga estratégias breves de regulação emocional e reforço positivo.
            """
        ),

        "redator_final": Agent(
            name="redator_final",
            model=MODEL_ID,
            description="Resume de forma clara, empática e concisa.",
            instruction="""
                Recebe as respostas dos especialistas e:
                • Cria um texto final coeso de no máx. 4 parágrafos curtos.
                • Lista (em tópicos) 3 referências ou links úteis ao final.
            """
        ),
    }

agentes = build_agents()   # cria todos



#### Função Principal

In [51]:
def gerar_resposta(topic: str) -> str:
    """Dispara os 5 especialistas e depois o redator."""
    especialistas = ["assistente_social", "juridico",
                     "educacional", "nutricionista", "psicologo"]

    respostas = []
    for chave in especialistas:
        prompt = f"Tema: {topic}\nElabore orientação dentro da sua especialidade."
        respostas.append(f"{chave}:\n" + call_agent(agentes[chave], prompt))

    # Junta tudo para o redator
    material = "\n\n".join(respostas)
    resumo = call_agent(agentes["redator_final"], material)
    return resumo

#### Fluxo de interação

In [52]:
tema_usuario = input("❓ Por favor, digite o TÓPICO relacionado a TEA que gostaria de pesquisar: ").strip()


while not tema_usuario:
    print("⚠️ Você esqueceu de digitar o tópico!")
    tema_usuario = input("digite o TÓPICO relacionado a TEA que gostaria de pesquisar:  ").strip()

print(f"✅ OK! Vamos então criar a pesquisa sobre {tema_usuario}")
resultado = gerar_resposta(tema_usuario)
to_markdown(resultado)


❓ Por favor, digite o TÓPICO relacionado a TEA que gostaria de pesquisar: uso de animais na terapia
✅ OK! Vamos então criar a pesquisa sobre uso de animais na terapia


> A Terapia Assistida por Animais (TAA) é uma abordagem terapêutica promissora para pessoas com Transtorno do Espectro Autista (TEA), oferecendo benefícios significativos nas áreas social, emocional e física. A interação com animais facilita a comunicação, reduz a ansiedade em situações sociais e estimula importantes sensações táteis e visuais. Profissionais qualificados, como terapeutas ocupacionais e psicólogos, podem conduzir a TAA, adaptando as atividades às necessidades individuais de cada criança e garantindo um ambiente seguro e controlado.
> 
> Para acessar a TAA, é essencial uma avaliação profissional que indique a terapia como parte de um plano terapêutico individualizado. Existem diversas instituições e ONGs que oferecem TAA, e os Centros de Referência de Assistência Social (CRAS) e as Associações de Pais e Amigos dos Autistas (APAA) podem fornecer informações e encaminhamentos. Além disso, o Projeto de Lei 4711/2023 propõe incluir a TAA no Sistema Único de Saúde (SUS), buscando facilitar o acesso a essa terapia para pessoas com autismo.
> 
> No contexto da alimentação, a TAA pode ser utilizada para criar associações positivas com novos alimentos e reduzir a ansiedade durante as refeições. A presença de um animal pode tornar o ambiente mais relaxante e incentivar a criança a experimentar novos alimentos, desde que supervisionado por um nutricionista especializado em TEA. O bem-estar do animal também é crucial, garantindo que seja treinado, seguro e adequado para interagir com crianças com TEA.
> 
> É importante lembrar que a TAA é uma ferramenta complementar e não substitui o acompanhamento profissional especializado. Cada caso é único, e a abordagem deve ser adaptada às necessidades e preferências individuais, respeitando os limites da pessoa com TEA e do animal.
> 
> **Referências e links úteis:**
> 
> *   Lei nº 12.764/2012 (Lei Berenice Piana): [https://www.planalto.gov.br/ccivil_03/_ato2011-2014/2012/lei/l12764.htm](https://www.planalto.gov.br/ccivil_03/_ato2011-2014/2012/lei/l12764.htm)
> *   Projeto de Lei 4711/2023: [https://www.camara.leg.br/proposicoesWeb/fichadetramitacao?idProposicao=2349510](https://www.camara.leg.br/proposicoesWeb/fichadetramitacao?idProposicao=2349510)
> *   Associações de Pais e Amigos dos Autistas (APAAs): (pesquisar a APAE da sua cidade)